# Main notebook

### Preprocessing

In [1]:
import pandas as pd 
import json 
import os
import random
from tqdm import tqdm 
import pickle 
from collections import defaultdict
from collections import Counter
from functions import *
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt 
from collections import Counter

In [ ]:
"""
Renaming the columns dataset
"""
df = pd.read_csv("wikigraph_reduced.csv", sep='\t') 
df.columns = ["Edges",'Source', 'Target']

In [ ]:
df.head(5)

In [ ]:
"""
save renamed dataset into a csv file
"""
df.to_csv('data/dataset.csv', index=False)

In [ ]:
"""
Taking as input the file wiki-topcats-page-names.txt, return a Dictionary with key the number of 
the page and as value the name of the page. 
"""
# keys: number of the page (article)
# values: name of the page (article)
p = open("wiki-topcats-page-names.txt", "r")
pages = {}
for pag in tqdm(p): 
    list_ = pag.split()[1:]
    aux = ' '.join(list_)
    pages[int(pag.split()[0])] = aux


In [ ]:
write_pickle('data/pages.pkl', pages)

In [ ]:
pages = read_pickle('data/pages.pkl')

In [ ]:
"""
Return the dictionary where as key the name of the category and 
as value the number of the articles that appears in the category. 
"""
# keys: category
# values: [list of the number of the pages (article)]
cat = open("wiki-topcats-categories.txt", "r")
categories = {}
for i in tqdm(cat): 
    category = i.split()[0][9:-1]
    page_in_cat = list(map(int, i.split()[1:])) 
    categories[category] = page_in_cat


In [ ]:
write_pickle('data/categories.pkl', categories)

In [ ]:
categories = read_pickle('data/categories.pkl')

In [ ]:
"""
each pages has a list of the names of the categories wich are linked to
"""
# keys: number of the page (article)
# values: [list of categories]
cat = open("wiki-topcats-categories.txt", "r")
cat_per_pages = {}
for i in tqdm(cat):
    category = i.split()[0][9:-1]
    page_in_cat = list(map(int, i.split()[1:]))

    if len(page_in_cat) >5000 and len(page_in_cat) < 30000:
        for x in page_in_cat:
            if x not in cat_per_pages:
                a= []
                a.append(category)
                cat_per_pages[x]=a
            else: 
                aux = list(cat_per_pages[x])
                aux.append(category)
                cat_per_pages[x]= aux

In [ ]:
"""
reducing cat_per_pages dictionary 
to have the condition that only one page is linked to only one category. 

"""
# keys: number of the page (article)
# values: category chosen at random and referring to that page
one_cat_per_pages = {}
for key, elem in tqdm(cat_per_pages.items()):
    one_cat_per_pages[key] = random.choices(elem, k = 1)


In [ ]:
write_pickle('data/one_cat_per_pages.pkl', one_cat_per_pages)

In [ ]:
one_cat_per_pages = read_pickle('data/one_cat_per_pages.pkl')

In [ ]:
"""
Finally, we create a last dictionary that takes us back to the initial state, i.e. 
the name of the category and the pages (integers) in that category. 
This time, each page is linked to a single category.
"""
# keys: category chosen at random 
# values: number of the pages (article) referring to the category
categories_red = {}
for key, elem in tqdm(one_cat_per_pages.items()):

    if elem[0] not in categories_red:
            a= []
            a.append(key)
            categories_red[elem[0]]=a
    else: 
            aux = list(categories_red[elem[0]])
            aux.append(key)
            categories_red[elem[0]]= aux


In [ ]:
write_pickle('data/categories_red.pkl', categories_red)

In [ ]:
categories_red = read_pickle('data/categories_red.pkl')

# RQ1

In [ ]:
data = pd.read_csv('data/dataset.csv')

Creation of the Graph

In [ ]:
graph = get_graph_dictionary(data)

In [ ]:
write_pickle('data/graph.pkl', graph)

In [ ]:
graph = read_pickle('data/graph.pkl')

In [ ]:
g = Graph(graph)

### Is the graph directed?

In [ ]:
"""
TO DO: review- Kernel Stopped 
Initialize the Graph
"""
import networkx as nx
import scipy as sp

a = (g.edges())
G = nx.DiGraph()
G.add_nodes_from(g.edges())
nx.is_directed()

In [ ]:
A = nx.to_scipy_sparse_matrix(G)

In [ ]:
def IsSymmetric(mat):
    """
    Build a lil matrix to create a sparse matrix of the vertices and edges,
    get the sum of the point in the matrix,
    check if the matrix is symmetric or not
    """
    mat = sp.lil_matrix((max(g.vertices())+1,max(g.vertices())+1), dtype=int)
    # looping on each vertex to assign the edges == 1
    for vertex in g.graph_d:   
        for target in g.graph_d[vertex]:
            mat[vertex, target] = 1
    
    rows, cols = mat.nonzero() # get only the non zero elements from the sparse matrix 
    return rows, cols

In [ ]:
# method to check if the matrix is symmetric
rows, cols = IsSymmetric(mat)
if np.cumsum((mat[cols, rows] == mat[rows, cols]).A)[-1] == mat[cols, rows].shape[1]:
    print('Is symmetric')
else:
    print('No symmetric')

### How many articles are we considering?

In [ ]:
print("Aricles of the graph:")
print(len(g.vertices()))

### How many hyperlinks between pages exist?

In [ ]:
print("Hyperlinks of the graph:")
print(len(g.edges()))

### Compute the average number of links in an arbitrary page. What is the graph density? Do you believe that the graph is dense or sparse? Is the graph dense?

In [ ]:
"""
prendo 100 pagine e calcolo la media del numero di link per pagina
"""
list_ = []
for i in range(100):
    list_.append(average_number_pages1(g))
    avg = sum(list_)/len(list_)
print(avg)

Here we compute the density of the graph. 
Since our graph is directed, we can compute the density as follows:
    $$D = \frac{\lvert{E}\rvert}{2\binom{\lvert{V}\rvert}{2}} = \frac{\lvert{E}\rvert}{\lvert{V}\rvert(\lvert{V}\rvert - 1)}$$

In [ ]:
density_graph(g)

From the results, we may say that our graph is sparse because the density is close to 0.

### Visualize the nodes' degree distribution

#### In degree Distribution 

In [ ]:
concat = [data['Source'], data['Target']]
df_concat = pd.concat(concat)

In [ ]:
all_nodes = list(df_concat.unique())
d_aux = dict.fromkeys(all_nodes, 0)
only_target_node = list(data.Target)
for node in tqdm(only_target_node):
    d_aux[node] +=1

In [ ]:
in_deg = Counter(sorted(list(d_aux.values())))
y_in = np.array(list(in_deg.values()))
y_in = y_in/len(g.vertices())
x_in = list(in_deg.keys())

In [ ]:
plt.figure(figsize=(13,6))
plt.bar(x_in , y_in,
        color=(0.2, 0.4, 0.6, 0.6),
        edgecolor='blue')
plt.title("In Degree distribution of the firts 50 nodes ")
plt.xlabel("In-degree")
plt.ylabel("Probability of node with in-degree = k ")
plt.xlim(-1, 60)
plt.show()

#### Out-Degree

In [ ]:
out_deg_list = []
for key,items in tqdm(g.graph_d.items()):
    if isinstance(g.graph_d[key], list):
         out_deg_list.append(len(g.graph_d[key]))
    elif (isinstance(g.graph_d[key], int)): 
        out_deg_list.append(1) 
out_deg = Counter(sorted(out_deg_list))
x_out = list(out_deg.keys())
y_out =  np.array(list(out_deg.values()))/ len(g.vertices())

In [ ]:
"""
ho troncato la distribuzione a 50 perchè andava troppo oltre e i valori erano tutti 0. 
"""
plt.figure(figsize=(13,6))
plt.bar(x_out, y_out,
        color=(0.2, 0.4, 0.6, 0.6),
        edgecolor='blue')
plt.title("Out Degree distribution of the firts 50 nodes ")
plt.xlabel("Out-degree")
plt.ylabel("Probability of node with out-degree = k ")
plt.xlim(-1, 50)
plt.show()

# RQ2

In [ ]:
from collections import defaultdict

dic = defaultdict(list)  # Creating default dictionary to store Source as key and Value as Target
for key,value in zip(data['Source'],data['Target']): 
    dic[str(key)].append(str(value))
    

In [ ]:
def pages(page, click):
    total_pages = [] # This list will store number of pages
    page_list = [] #This list will store input vlaue initially and then will add correspondence value as per number of click
    page_list.append(str(page))
    for no_of_click in range(click): #This will run as per number of clicks
        new_lst = []                 
        for i in page_list:
            for j in dic[i]:
                new_lst.append(str(j))
                total_pages.append(str(j))
        page_list = new_lst
    return total_pages

In [ ]:
try:
    page = input(" Enter page number ")
    click = input(" Enter number of clicks ")
    if page!='' or click!='':
        total_pages = pages(page,int(click))
        if len(total_pages)!=0:
            print("User can reach {} pages after {} clicks".format(len(total_pages),click))
        else:
            print("There is no link for this page, Kindly try with another page")
    else:
        print(" *********** Kindly provide valid input  **********")
except ValueError:
    print("No valid input! Please try again ...")

# RQ3

Define a function that takes in input:
- A category C
- A set of pages in C, p = {p1, ..., pn}

and returns the minimum number of clicks required to reach all pages in p, starting from the page v, corresponding to the most central article, according to the in-degree centrality, in C.

In [ ]:
def minimum_number_clicks(graph, categories_red, data):
    print('Write the category')
    while True:
        category_input = str(input())
        if category_input not in categories_red:
            print(category_input, ' not exist as category, change category')
        else:
            break
    print()
    print("Write the set of pages in the category chosen separated by a ','")
    print()
    pages_input = input()
    pages_input = pages_input.split(',')
    pages_input = [int(i) for i in pages_input]

    pages_not = []
    for pages in pages_input:
        if pages not in categories_red[category_input]:
            print(pages, ' not in ', category_input)
            pages_not.append(pages)
    pages_input = [i for i in pages_input if i not in pages_not]  
    
    graph = g.graph_d                    # the graph
    central_vertex = most_central_article(categories_red[category_input], in_degree_centrality(data))[0]   # set the max vertex
    v = central_vertex
    visited = [False] * (max(graph) + 1) # set as False the visited vertex
    queue = []                           # set the queue list
    queue.append(v)                      # append the starting vertex to the list
    queue_complete = []                  # set the complete list to 0 
    visited[v] = True                    # set the starting vertex as visited
    reached = 0                          # initialize the number of reached vertex
    reached_vertex = []                  # initialize the list of reached vertex
    number_of_click = 0

    while queue:
        if reached < (len(pages_input)):
            v = queue.pop(0)
            #print(v, end=' ')

            try:
                number_of_click += 1
                for i in graph[v]:
                    if visited[i] == False:
                        visited[i] = True
                        queue.append(i)
                        queue_complete.append(i)
                        if i in pages_input:
                            reached += 1
                            reached_vertex.append(i)
                            #print('Vertex', i, 'reached')
            except TypeError:
                number_of_click += 1
                j = graph[v]
                if visited[j] == False:
                    visited[j] = True
                    queue.append(j)
                    queue_complete.append(j)
                    if j in pages_input:
                        reached += 1
                        reached_vertex.append(j)
                        #print('Vertex', i, 'reached')

        else:
            break
    print('Reached vertex are: {}'.format(reached_vertex))
    print('Minimum number of clicks, from most central article {} to reach the set of pages, is {}.'.format(central_vertex, number_of_click))
    not_reached_vertex = [i for i in pages_input if i not in reached_vertex]
    print('Not possible to reach {}'.format(not_reached_vertex))

In [ ]:
minimum_number_clicks(g.graph_d, categories_red, data)

In [ ]:
count=0
for el in categories_red['Living_people']:
    if el in q:
        print(el)
    count+=1
    if count == 10000:
        print()
        print('-'*20, count, '-'*20)
        print()
print(count)

# RQ4

In this question we were asked to implement a function that, given two categories as input, would return the subgraph induced by all the pages belonging to those two categories.

Our reasoning in creating the function below was that the best course of action would be to consider all those pages that could be found in the "source" and "target" columns of the main dataset, that being ds. In this way, we could avoid selecting many of those pages found in these categories that have no links to any other page. So in the cat_subgraph function we firstly selected all the pages belonging to the two categories and then created a subset of the main dataset, taking into account only the pages belonging to edges that either connect the two categories or are within the categories themselves.

In [ ]:
sub_graph = cat_subgraph('Main_Belt_asteroids', 'Asteroids_named_for_people', ds)
sub_graph

Once the structure of the subgraph is computed as a dictionary, we can apply the Graph class to it.

In [ ]:
sub_g = Graph(sub_graph)

Finally, we can visualise the first order neighbours of a given node, as shown below:

In [ ]:
G = nx.Graph()
G.add_edges_from(sub_g.edges())
show_first_order_neigbors(G, start_node=865446)

#### second function

For the second question of this request, we created two additional functions. The first one finds all the hyperlinks that connect the two nodes given as input, within the subgraph of their categories.

It must be noted that, because of the preprocessing, not every pair of nodes has a meaningful output for this function. This is because many paths are missing in our dataframe, so that it is not possible to go from any node to any other node in the way we would in the complete Wikipedia. In fact, we observed that most of the nodes that have links are actually part of clusters of nodes, that is group of nodes that are connected to each other through paths. One such cluster is found as the output of this function.

In [ ]:
find_hyperlinks(sub_graph, 865445, 865449)

The second function returns the amount of paths found, which comprises the minimum number of links that need to be cut in order to disconnect the two nodes.

In [ ]:
min_hyperlinks(sub_graph, 865445, 865449)

# RQ5

For the fifth question we had to write a function that would return the distance between a category given as input and every other category in our dataframe.

As previously stated, because of the preprocessing and limitedness of the dataframe, many of the paths between nodes are missing, so that it is not possible to draw a complete path between any pair of nodes. For this reason, in the final output many distances are missing.

# how do we handle these exceptions?

In [ ]:
concat = [ds['Source'], ds['Target']]
df_concat = pd.concat(concat)
all_nodes = list(df_concat.unique())

In [ ]:
def distances_from_category(ic, cat, all_nodes):
    results = {}

    for c in tqdm(cat.keys()):
        if c != ic:
            visited = dict.fromkeys(all_nodes, False)
            distance = dict.fromkeys(all_nodes, float('inf'))
            sub_graph = cat_subgraph(c, ic, ds)
            for key,val in sub_graph.items():
                if isinstance(sub_graph[key], int):
                    sub_graph[key] = [sub_graph[key]]
            pages = relevant_pages(c, ds)
            aux = []
            for i in pages:
                x = page_distance(i, ic, ds, sub_graph, visited, distance)
                if x:
                    aux.append(x)
            merged = np.array(list(itertools.chain(*aux)))
            m = np.median(merged)
            results[c] = m
            
    write_pickle('data/' + ic, results)
    return (results)

The output of the function consists of the remaining categories sorted in order of closeness to the category given as input.

In [ ]:
distances_from_category('Main_Belt_asteroids', cat, all_nodes)

# RQ6

In [ ]:
len(categories_red['Buprestoidea'])

In [ ]:
data[data['Target'] == 1185516].Source.values.tolist()

In [ ]:
concat = [data['Source'], data['Target']]    # concat all the nodes
all_nodes = list(pd.concat(concat).unique()) 
inbound = {}
for node in all_nodes:
    inbound[node] = data[data['Target'] == node].Source.values.tolist()
#for _,row in data.iterrows():
#    inbound[row['Target']].append(row['Source'])

In [ ]:
normalized_vertex = dict.fromkeys(categories_red['Buprestoidea'], 1/len(categories_red['Buprestoidea'])) 

In [ ]:
normalized_vertex

In [ ]:
inbound

In [ ]:
max_iterations = 2
for i in max_iterations:
    normalized_vertex_temp = dict.fromkeys(categories_red['Buprestoidea'], 0) 
    for vertex in normalized_vertex:
        pr = 0
        for in_bound in inbound[vertex]:
            pr += normalized_vertex[inbound]/out_degree_centrality(g)[inbound]
        
        normalized_vertex_temp[vertex] = pr
    
    normalized_vertex = normalized_vertex_temp

In [ ]:
inbound[301]

In [ ]:
out_degree_centrality(g)[0]